In [1]:
import pandas as pd
import numpy as np
import snowflake.connector

In [6]:
#login to snowflake db
con = snowflake.connector.connect(user='vishal.kumar@scale.com',
                                 account='pxa65918',
                                 authenticator='externalbrowser',
                                 warehouse='COMPUTE_WH',
                                 database='SCALE_CRAWLER',
                                 role='GENERAL_RO')

cs = con.cursor()

Initiating login request with your identity provider. A browser window should have opened for you to complete the login. If you can't see it, check existing browser windows, or your OS settings. Press CTRL+C to abort and try again...


In [13]:
#Get work logs
sql = f'''
select
    brand,
    pvid,
    batch,
    preset_attributes:category::string as category,
    to_varchar(attributes_entries) as attributes,
    regexp_count(attributes, '\\b["Cannot Be Determined"]\\b', 1) as cbds,
    length(scraped_attributes:description) as desc_len,
    length(scraped_attributes:title) as title_len,
    desc_len + title_len as total_len
from
    public.productvariants
where
    pvid ilike '%walmart%'
    and cast(created_at as Date) > '2023-06-10'
having
    length(attributes) > 2
order by
    cbds asc
'''
cs.execute(sql)
worklogdf = cs.fetch_pandas_all()


In [14]:
print(worklogdf)

                  BRAND                        PVID  \
0       www.walmart.com   www.walmart.com!782253389   
1       www.walmart.com   www.walmart.com!622732932   
2       www.walmart.com   www.walmart.com!476539841   
3       www.walmart.com   www.walmart.com!294781700   
4       www.walmart.com  www.walmart.com!1875725474   
...                 ...                         ...   
147714  www.walmart.com   www.walmart.com!369834916   
147715  www.walmart.com   www.walmart.com!314678438   
147716  www.walmart.com      www.walmart.com!446516   
147717  www.walmart.com   www.walmart.com!336417154   
147718  www.walmart.com    www.walmart.com!51793576   

                                       BATCH                      CATEGORY  \
0                   wave-3.0//Media_20230530                         Books   
1                    wave-2.2//Home_20230530           Curtains _ Valances   
2                   wave-3.0//Media_20230530                         Books   
3                   wave-3.

In [12]:
#save worklog to csv
worklogdf.to_csv('CorrelationAnalysis.csv', index=False)